# Comparison against First Best

The goal here is to compare both the traditional and mechanism-implemented game against the first best option.

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import math
import random
import itertools
from typing import List, Dict
import matplotlib.pyplot as plt
import seaborn as sns
from collections import namedtuple, defaultdict
from pandas.plotting import parallel_coordinates
# from PSSimPy.simulator import ABMSim
from PSSimPy import Bank, Account
from PSSimPy.credit_facilities import AbstractCreditFacility
from PSSimPy.utils import add_minutes_to_time, minutes_between

from MDP import MechMDPSearch, MDPStateExt, TradMDPSearch, MDPStateTrad
from SimClasses import ABMSim

## Setup

In [ ]:
class FirstBestBank(Bank):

    def cheapest_feasible_channels(self):
        options = []

        if self.posted_collateral > 0:
            options.append(("trad", gamma, self.posted_collateral, gamma))

        unpledged = sum(tx.amount for tx in self.txn_in
                        if tx.status_code == 0 and not tx.pledged)
        if unpledged > 0:
            options.append(("claim", phi, unpledged, phi))

        options.append(("unsec", chi, float('inf'), chi))
        options.append(("delay", delta, float('inf'), delta))

        # sort by marginal cost
        return sorted(options, key=lambda x: x[3])

    def strategy(self, txns_to_settle, *_):
        if not txns_to_settle:
            return set()

        obligation = sum(tx.amount for tx in txns_to_settle)
        shortfall  = max(0.0, obligation - self.balance)

        # Greedy covering of shortfall
        for ch_name, ch_cost, capacity, _ in self.cheapest_feasible_channels():
            if ch_name == "delay":               # cheapest is already delay
                return set()                     # planner delays whole batch

            take = min(capacity, shortfall)
            if take > 0:
                # account bookkeeping
                if ch_name == "trad":
                    self.posted_collateral -= take
                elif ch_name == "claim":
                    amount_left = take
                    for tx in self.txn_in:
                        if tx.status_code == 0 and not tx.pledged:
                            pledge = min(amount_left, tx.amount)
                            tx.pledged = True
                            amount_left -= pledge
                            if amount_left == 0:
                                break
                # unsecured needs no inventory
                self.balance += take
                shortfall   -= take

            if shortfall == 0:
                return txns_to_settle

        # still shortage after exhausting all credit => delay
        return set()
